<a href="https://colab.research.google.com/github/cianc/AoC2023/blob/main/day05_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
TEST = False #@param {type: "boolean"}
TEST_INPUT = 'day5_test1.txt'
INPUT = 'day5_input1.txt'
PART=2 #@param {type: "integer"}


In [ ]:
import re
import tqdm
from dataclasses import dataclass

@dataclass
class SupplyMapping:
  src_interval: tuple[int, int]
  dst_interval: tuple[int, int]

def get_input() -> list[str]:
  input = TEST_INPUT if TEST else INPUT
  with open(input, 'r') as f:
    rows = f.read().splitlines()
  return rows

def parse_seeds(row: str) -> tuple[int, list[tuple[int, int]]]:
  seeds_regex = r'^seeds: (( *\d+ *)+)$'
  m = re.match(seeds_regex, row)
  seeds = [int(s) for s in m.group(1).split()]
  seed_intervals = []
  seed_count = 0
  for i in range(0, len(seeds) - 1, 2):
    num_seeds = seeds[i+1] - 1
    seed_count += num_seeds
    seed_intervals.append((seeds[i], seeds[i] + num_seeds))

  return seed_count, seed_intervals


def parse_almanac(rows: list[str]) -> dict[tuple[str, str], list[SupplyMapping]]:
  supply_mappings = {}
  category_name_regex = r'^([a-zA-Z]+)-to-([a-zA-Z]+) map:$'
  category_map_regex = r'^(( *\d+ *)+)$'
  row_index = 0
  while row_index < len(rows):
    m = re.match(category_name_regex, rows[row_index])
    row_index +=1
    if not m:
      continue

    src_name = m.group(1)
    dst_name = m.group(2)
    supply_mappings.setdefault((src_name, dst_name), [])

    while row_index < len(rows):
      m = re.match(category_map_regex, rows[row_index])
      row_index +=1
      if not m:
        break

      dst_start, src_start, range_length = [int(m) for m in m.group(1).split()]
      supply_mappings[(src_name, dst_name)].append(
          SupplyMapping((src_start, src_start+range_length-1),
                          (dst_start, dst_start+range_length-1)))


  return supply_mappings


def calc_mappings(
    match_intervals: list[tuple[int, int]], mapping: SupplyMapping) -> tuple[list[tuple [int, int]], list[tuple[int, int]]]:

  destinations = []
  remainders = []
  interval2 = mapping.src_interval
  for interval1 in match_intervals:
    # No overlap
    if interval1[1] < interval2[0] or interval1[0] > interval2[1]:
      remainders.append(interval1)
      continue

    if interval1[0] <= interval2[0]:
      overlap_start = interval2[0]
    else:
      overlap_start = interval1[0]

    if interval1[1] <= interval2[1]:
      overlap_end = interval1[1]
    else:
      overlap_end = interval2[1]

    if overlap_start > interval1[0]:
      remainders.append((interval1[0], overlap_start-1))
    if overlap_end < interval1[1]:
      remainders.append((overlap_end+1, interval1[1]))

    mapping_offset = mapping.dst_interval[0] - mapping.src_interval[0]
    destinations.append(
        (overlap_start + mapping_offset, overlap_end + mapping_offset))

  return destinations, remainders

def min_location(
    seed_intervals: list[tuple[int, int]],
    supply_mappings: dict[tuple[str, str], list[SupplyMapping]]) -> int:

  src_intervals = seed_intervals
  for mapping_key in (
      ('seed', 'soil'), ('soil', 'fertilizer'), ('fertilizer', 'water'),
      ('water', 'light'), ('light', 'temperature'),
      ('temperature', 'humidity') ,('humidity', 'location')):
    supply_mapping = supply_mappings[mapping_key]
    destination_intervals = []
    remainders = src_intervals
    for mapping in supply_mapping:
      if not remainders: break
      destinations, remainders = calc_mappings(
          remainders, mapping)

      destination_intervals.extend(destinations)

    src_intervals = destination_intervals + remainders
  return min([s[0] for s in src_intervals])

input = get_input()
num_seeds, seeds = parse_seeds(input[0])
supply_mappings = parse_almanac(input[1:])
print(min_location(seeds, supply_mappings))


